Descript Audio Codec (.dac): High-Fidelity Audio Compression with Improved RVQGAN

In [1]:
import dac
from audiotools import AudioSignal

# Download a model
model_path = dac.utils.download(model_type="44khz")

model = dac.DAC.load(model_path, device = "cuda")


/home/lekhab/.conda/envs/codec_fake/lib/python3.12/site-packages/audiotools/ml/layers/base.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(loca

In [ ]:
import os


current_dir = os.path.dirname(__file__)

# Load audio signal file
file_path = os.path.join(current_dir, "sample_9.wav")
signal = AudioSignal(file_path)

# Encode audio signal as one long file
# (may run out of GPU memory on long files)
signal.to(model.device)

x = model.preprocess(signal.audio_data, signal.sample_rate)
z, codes, latents, _, _ = model.encode(x)

In [ ]:
import dac
from audiotools import AudioSignal
import os

# Download a model
model_path = dac.utils.download(model_type="44khz")

model = dac.DAC.load(model_path, device = "cuda")

current_dir = os.path.dirname(__file__)

# Load audio signal file
file_path = os.path.join(current_dir, "sample_9.wav")
signal = AudioSignal(file_path)

# Encode audio signal as one long file
# (may run out of GPU memory on long files)
signal.to(model.device)

x = model.preprocess(signal.audio_data, signal.sample_rate)
z, codes, latents, _, _ = model.encode(x)

# Decode audio signal
y = model.decode(z)

# Alternatively, use the `compress` and `decompress` functions
# to compress long files.

# signal = signal.cpu()
# x = model.compress(signal)

# # Save and load to and from disk
# x.save("compressed.dac")
# x = dac.DACFile.load("compressed.dac")

# # Decompress it back to an AudioSignal
# y = model.decompress(x)

# Write to file
# y.write('output.wav')

c:\Users\bolli\anaconda3\Lib\site-packages\audiotools\ml\layers\base.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(location, "cpu")
c:\Users\

In [8]:
latents.shape

torch.Size([1, 72, 862])

In [ ]:
codes.shape

: 

In [2]:
#to check if torch has cuda

import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


True
11.8


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

: 